In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding,CuDNNGRU
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [ ]:
dataset = pd.read_csv('../input/hepsiburada.csv')

Datasetimizi inceleyelim

In [ ]:
dataset

Datasetimizde 2 kısımdan oluşuyor 1. kısım yorumların iyi/kötü olduğunu belirleyen 1/0 lar 2. kısım ise kullanıcıların yorumları

In [ ]:
target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()

Test ve train kısmına ayırdık

In [ ]:
seperate = int(len(data)/0.8)
x_train , x_test = data[:seperate], data[seperate:]
y_train , y_test = target[:seperate], target[seperate:]

Tokenleştirme için kelime sayısını belirlemeliyiz.

In [ ]:
limit = 10000
tokenizer = Tokenizer(num_words =limit)
tokenizer.fit_on_texts(data) ###tokenleştirme#

In [ ]:
tokenizer.word_index ### en çok kullanılan 10000 kelimeyi tokenleştirdi.###

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train) ### cümleleri bir token haline dönüştürdük###

In [ ]:
x_train[800]

In [ ]:
print(x_train_tokens[800])

In [ ]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

RNN işlemlerinde eşit boyut olmasını sağlamalıyız bunun için padding yapıyoruz. Padding yapacağımız sayı aralığını belirlemeliyiz.

In [ ]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens ]
num_tokens = np.array(num_tokens)
max_tokens = np.mean(num_tokens) + 2*np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

In [ ]:
x_train_pad = pad_sequences(x_train_tokens,maxlen = max_tokens)

In [ ]:
x_test_pad = pad_sequences(x_test_tokens,maxlen = max_tokens)

In [ ]:
x_train_pad[800]

Tokenleri tekrardan kelimelere çevirebilmek için fonksiyon oluştur


In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [ ]:
def tokens_to_string(tokens):
    words=[inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [ ]:
x_train[800]

In [ ]:
tokens_to_string(x_train_tokens[800])

In [ ]:
model = Sequential()
embedding_size = 50 # her kelimeye karşılık gelen 50 uzunluğundaki vektör

In [ ]:
model.add(Embedding(input_dim=limit,output_dim=embedding_size,input_length=max_tokens,name='embedding_layer'))

In [ ]:
model.add(CuDNNGRU(units=16,return_sequences=True))
model.add(CuDNNGRU(units=8,return_sequences=True))
model.add(CuDNNGRU(units=4))
model.add(Dense(1,activation='sigmoid'))

In [ ]:
optimizer = Adam(lr=1e-3)

In [ ]:
model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x_train_pad, y_train,epochs=5,batch_size=256)

Kaynak : https://www.udemy.com/dogal-dil-isleme